# Анализ сервиса для чтения книг по подписке

**Цель исследования**: Провести анализ базы данных. Результаты помогут сформулировать ценностное предложение нового продукта.

**Описание данных**: Данные для анализа содержатся в 5 таблицах.
1. `books` содержит данные о книгах:
    - `book_id` — идентификатор книги;
    - `author_id` — идентификатор автора;
    - `title` — название книги;
    - `num_pages` — количество страниц;
    - `publication_date` — дата публикации книги;
    - `publisher_id` — идентификатор издателя.
    
    
2. `authors` содержит данные об авторах:
    - `author_id` — идентификатор автора;
    - `author` — имя автора.
    
    
3. `publishers` Содержит данные об издательствах:
    - `publisher_id` — идентификатор издательства;
    - `publisher` — название издательства;
    
    
4. `ratings` содержит данные о пользовательских оценках книг:
    - `rating_id` — идентификатор оценки;
    - `book_id` — идентификатор книги;
    - `username` — имя пользователя, оставившего оценку;
    - `rating` — оценка книги.
    
    
5. `reviews` содержит данные о пользовательских обзорах на книги:
    - `review_id` — идентификатор обзора;
    - `book_id` — идентификатор книги;
    - `username` — имя пользователя, написавшего обзор;
    - `text` — текст обзора.
    
    
    
**Ход исследования**

1. Проведем обзор данных
2. Выполним 5 заданий:
    2.1. Посчитать, сколько книг вышло после 1 января 2000 года;
    2.2. Для каждой книги посчитать количество обзоров и среднюю оценку;
    2.3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры);
    2.4. Определить автора с самой высокой средней оценкой книг — учесть только книги с 50 и более оценками;
    2.5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
    
3. Подведем итоги

## 1. Обзор данных

Импортируем библиотеки:

In [1]:
import pandas as pd
import sqlalchemy as sa
pd.set_option('display.max_colwidth', False)

Устанавливаем параметры:

In [2]:
db_config = {
    'user': '', # имя пользователя
    'pwd': '', # пароль
    'host': '',
    'port': , # порт подключения
    'db': '' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

#сохраняем коннектор:
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

Чтобы выполнить SQL-запрос, напишем функцию с использованием Pandas `get_sql_data`:

In [3]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, 
    получаем данные из sql, 
    закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Для вывода первых пяти строк и информации о таблицах напишем функцию `read_sql_tabl`

In [4]:
def read_sql_tabl(df):
    '''Функция для вывода 
    5 первых строк и общей
    информации о таблице
    Args:
        df: SQL запрос
    для вывода таблицы
    Returns:
        5 строк pd.DataFrame
        общая информация pd.DataFrame.
    '''    

    display(pd.io.sql.read_sql(sql=sa.text(df), con = engine).head())
    pd.io.sql.read_sql(sql=sa.text(df), con = engine).info()


### 1.1. Обзор таблицы `books`

Используем функцию `read_sql_tabl` и выведем информацию о таблице `books`

In [5]:
read_sql_tabl('''SELECT * FROM books''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


В таблице `books` 6 столбцов и 1000 строк. 4 столбца с числами и 2 с типом object, один из которых содержит даты. Объем памяти 47.0 KB. Пропуски в данных отсутствуют.

### 1.2. Обзор таблицы `authors`

Выгрузим данные об авторах.

In [6]:
read_sql_tabl('''SELECT * FROM authors''')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


В таблице `authors` 2 столбца и 636 строк. 1 столбец содержит числа, а 2-й с типом object содержит текст. Объем памяти 10.1 KB. Пропуски в данных отсутствуют.

### 1.3. Обзор таблицы `publishers`

Выгрузим данные об издательствах.

In [7]:
read_sql_tabl('''SELECT * FROM publishers''')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


В таблице `publishers` 2 столбца и 340 строк. 1 столбец содержит числа, а 2-й с типом object содержит текст. Объем памяти 5.4 KB. Пропуски в данных отсутствуют.

### 1.4. Обзор таблицы `ratings`

Выгрузим данные о пользовательских оценках книг.

In [8]:
read_sql_tabl('''SELECT * FROM ratings''')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


В таблице `ratings` 4 столбца и 6456 строк. 3 столбца содержит числа, 1 тип object содержит текст. Объем памяти 201.9 KB. Пропуски в данных отсутствуют.

### 1.5. Обзор таблицы `reviews`

Выгрузим данные о пользовательских обзорах на книги

In [9]:
read_sql_tabl('''SELECT * FROM reviews''')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


В таблице `reviews` 4 столбца и 2793 строки. 2 столбца содержит числа, 2 тип object содержат текст. Объем памяти 87.4 KB. Пропуски в данных отсутствуют.

### 1.6. Выводы

Для анализа предоставлено 5 таблиц:
- `books` 6 столбцов и 1000 строк.
- `authors` 2 столбца и 636 строк.
- `publishers` 2 столбца и 340 строк.
- `ratings` 4 столбца и 6456 строк.
- `reviews` 4 столбца и 2793 строки

Пропуски в данных отсутствуют.

## 2. Выполнение заданий

### 2.1. Посчитаем, сколько книг вышло после 1 января 2000 года.

Данные о книгах содержатся в таблице `books`, для расчета количества книг, выпущенных в указанный период, нам достаточно только этой таблицы:

In [10]:
query = (
    '''
    SELECT
        COUNT(title) 
    FROM books
    WHERE publication_date > '2000-01-01'
    '''
)
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг (81.9% от базы данных о книгах)

### 2.2. Для каждой книги посчитаем количество обзоров и среднюю оценку.

Данные об обзорах содержатся в таблице `reviews`, данные об оценках в таблице `ratings`, добавим их к таблице `books`, выведем на экран идентификатор книги, название книги, количество обзоров и средний рейтинг 

In [11]:
query = (
    '''
SELECT b.book_id, 
       b.title,
       COUNT(DISTINCT review_id) AS count_review,
       ROUND(AVG(rating), 2) AS avg_rating      
FROM books AS b
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY count_review DESC, avg_rating DESC
    '''
)
get_sql_data(query)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden Book),0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


- Больше всего обзоров на книгу `Twilight (Twilight #1)` - 7, при этом у нее средний уровень рейтинга 3.66 баллов.
- В ТОП-3 книг по количеству обзоров книги из серии о Гарри Поттере `Harry Potter and the Prisoner of Azkaban (Harry Potter #3)	` и `Harry Potter and the Chamber of Secrets (Harry Potter #2)`, у этих книг по 6 обзоров и самый высокий рейтинг среди книг с таким количеством обзоров.

### 2.3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (чтобы исключить из анализа брошюры).

Информация об издательствах содержится в таблице `publishers`. Добавим эту таблицу к таблице `books`.

In [12]:
query = (
    '''
SELECT publisher,
        COUNT(book_id) AS count_book

FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY count_book DESC      
LIMIT 1
    '''
)
get_sql_data(query)

,publisher,count_book
0,Penguin Books,42


Наибольшее число 42 книги толщиной более 50 страниц выпустило издательство `Penguin Books` 

### 2.4. Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками.

Данные об авторах содержатся в таблице `authors`, оценки содержатся в таблице `ratings`. Прямых связей между таблицами нет, объединим их через таблицу `books`

In [13]:
query = (
    '''
  
SELECT a.author,
       ROUND(AVG(rating), 2) AS avg_rating
FROM authors AS a
LEFT JOIN books AS b ON a.author_id = b.author_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
WHERE a.author_id in (SELECT b.author_id
                    FROM books AS b
                    LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
                    GROUP BY b.book_id
                    HAVING COUNT(rating) >= 50)
GROUP BY a.author_id
ORDER BY avg_rating DESC
LIMIT 5

    '''
)
get_sql_data(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.24
3,Louisa May Alcott,4.20
4,Rick Riordan,4.13


Среди книг с 50 и более оценками с самой высокой средней оценкой 4.29 автор  `J.K. Rowling/Mary GrandPré`. Дж. Роулинг и ее "Гарри Поттер" с иллюстрациями от Мари ГранПре. 

### 2.5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Данные об обзорах содержатся в таблице `reviews`, данные об оценках в таблице `ratings`

In [14]:
query = (
    '''
WITH
rt AS 
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating) > 48
    ORDER BY COUNT(rating) DESC),

rv AS
    (SELECT username,
           COUNT(review_id) AS count_review
    FROM reviews 
    GROUP BY username)
    

SELECT AVG(count_review)
FROM rt
JOIN rv ON rt.username = rv.username

    '''
)
get_sql_data(query)

,avg
0,24.0


Пользователи, поставившие более 48 оценок, оставили в среднем 24 обзора.

## 3. Итоги исследования


Мы проанализировали 5 таблиц в соответствии с ТЗ и получили следующую информацию:

1. После 1 января 2000 года вышло 819 книг (81.9% от базы данных о книгах)


2. Средни всех книг, содержащися в базе: 
    - Больше всего обзоров на книгу `Twilight (Twilight #1)` - 7, при этом у нее средний уровень рейтинга 3.66 баллов.
    - В ТОП-3 книг по количеству обзоров книги из серии о Гарри Поттере `Harry Potter and the Prisoner of Azkaban (Harry Potter #3)	` и `Harry Potter and the Chamber of Secrets (Harry Potter #2)`, у этих книг по 6 обзоров и самый высокий рейтинг среди книг с таким количеством обзоров.
    
    
3. Наибольшее число 42 книги толщиной более 50 страниц выпустило издательство `Penguin Books`.


4. Среди книг с 50 и более оценками с самой высокой средней оценкой 4.29 автор  `J.K. Rowling/Mary GrandPré`. Дж. Роулинг и ее "Гарри Поттер" с иллюстрациями от Мари ГранПре. 


5. Пользователи, поставившие более 48 оценок, оставили в среднем 24 обзора.

Рекомендации:

1. Для привлечения новых пользователей нужно увеличить книготеку, 1000 книг в подборке от 636 авторов - это небольшое количество. 
2. Пользователи оставляют небольшое количество обзоров и оценок книг, нужно внедрить инструменты, которые помогут пользователям более активно делиться своим мнением. В качестве инструмента поощрения за участие в рецензировании можно дарить подписку на чтение какой-то рейтинговой книги.
3. При формировании ценностного предложения учитывать количество обзоров и уровень рейтинга в совокупности. Можем сказать, в лидерах серии книг от Дж. Роулинг о Гарри Поттере, и от таких авторов как:
    - Маркус Зусак,
    - Дж.Р.Толкиен,
    - Луиза Мей Олкотт,
    - Рик Риордан.        